In [8]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np

In [2]:
df1 = pd.read_csv('../first_insurance_data.csv',index_col =0)

/home/ambuj/izenda_fullcycle/izenda_insurance_data/insurance_env/lib/python3.5/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [32]:
df1.shape

(1278568, 10)

In [34]:
df1.columns

Index(['LossYearMo', 'State', 'DivisionName', 'DisplayName', 'CommOrRes',
       'NoteCount', 'PhotoCount', 'JobCount', 'ClaimCount',
       'NewEstimateTotal'],
      dtype='object')

In [36]:
df1.nunique()

LossYearMo            12
State                 62
DivisionName        2486
DisplayName           17
CommOrRes            218
NoteCount           4492
PhotoCount          1857
JobCount              96
ClaimCount            62
NewEstimateTotal       2
dtype: int64

In [3]:
df1_50 = df1.sample(n=50000,random_state=42)

In [4]:
processed_df1 = pd.get_dummies(df1_50,sparse=True)

In [9]:
def train_validate_test_split(df, train_percent=.60, validate_percent=.20, seed=42):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [14]:
train, valid, test = train_validate_test_split(processed_df1.reset_index(drop=True))

In [15]:
train.shape, valid.shape, test.shape

((30000, 1549), (10000, 1549), (10000, 1549))

In [23]:
lr = LogisticRegression(random_state=42,verbose=1)

In [37]:
lr.fit(train.loc[:,train.columns != 'NewEstimateTotal'], train['NewEstimateTotal'])

/home/ambuj/izenda_fullcycle/izenda_insurance_data/insurance_env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=1, warm_start=False)

In [28]:
valid_preds = lr.predict(valid.loc[:,valid.columns != 'NewEstimateTotal'])

In [29]:
metrics.accuracy_score(valid.NewEstimateTotal, valid_preds)

0.7555

In [30]:
test_preds = lr.predict(test.loc[:,test.columns != 'NewEstimateTotal'])

In [31]:
metrics.accuracy_score(test.NewEstimateTotal, test_preds)

0.7575